In [1]:
##2b_Qapp_PrimaryCareDELTAtoStaging.ipynb - last modified 20230302 
## 20230302 - Added in srorganisation
## 20230208 - to account for thosepatients with no DOB etc as they are skeewing the data - they will be removed from the local persons table
## 20230209 - to account for persons not in obs period. Notes below 
#Builds the PrimaryCare FDM
#Assumes that the targetdb below exists
#Needs code to build it if it doesn't

#You will need to manually amend the target dataset for this script

library(bigrquery)
library(lubridate) # required for datetime

# Store the project ID
project_id = "yhcr-prd-phm-bia-core"

sourcedb <-'yhcr-prd-phm-bia-core.CB_MYSPACE_JDB'
targetdb <-'yhcr-prd-phm-bia-core.CB_STAGING_DATABASE_PrimaryCare'
    demo <-'yhcr-prd-phm-bia-core.CB_STAGING_DATABASE.src_DemoGraphics_MASTER'
targetdb <-gsub(' ','',targetdb)
print (sourcedb) 
print (targetdb)
print (demo)


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




[1] "yhcr-prd-phm-bia-core.CB_MYSPACE_JDB"
[1] "yhcr-prd-phm-bia-core.CB_STAGING_DATABASE_PrimaryCare"
[1] "yhcr-prd-phm-bia-core.CB_STAGING_DATABASE.src_DemoGraphics_MASTER"


In [3]:
#tbl_SRAppointment 

sql1 <-paste('delete from ', sourcedb,'.tbl_SRAppointment 
where digest is null ', sep = "")
tb1 <- bq_project_query(project_id, sql1)

sql2 <-paste('delete from ', sourcedb,'. tbl_SRAppointment 
where RowIdentifier like \'%---%\'; ', sep = "")
tb2 <- bq_project_query(project_id, sql2)

sql3 <-paste('insert into  ',targetdb,'.tbl_SRAppointment 
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
demo.person_id ,
IDOrganisationVisibleTo,
cast( DateStart as Datetime) as DateStart ,
cast(DateEnd as Datetime) as  DateEnd,
AppointmentStatus,
DateAppointmentBooked,
DatePatientArrival,
DatePatientSeen,
FollowUpAppointment,
TelephoneAppointment,
IDClinician,
IDProfileClinician,
RotaName,
RotaType,
RotaLocation,
RotaCode,
IDRotaOwner,
IDProfileRotaOwner,
AllowsOverbooking,
BookingContactNumber,
DateAppointmentCancelled,
IDRota,
IDReferralIN,
IDPatient,
apt.Digest,
IDOrganisation,
IDOrganisationRegisteredAt,
RemovedData,
cb_loadnum,
cb_exported_to_sde ,
Cast(cb_rowid as int64) ,
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SRAppointment apt
left join ', demo,' demo
on apt.Digest = demo.digest; ', sep = "")

tb3 <- bq_project_query(project_id, sql3)


eric <- paste("tbl_SRAppointment done "  ,Sys.time() + hours(1) ,sep = "")
print(eric) 


#tbl_SRCHStatusHistory


sql4 <-paste('delete from ', sourcedb,'.tbl_SRCHSStatusHistory where Digest is null', sep = "")
tb4 <- bq_project_query(project_id, sql4)

sql5 <-paste('insert into   ',targetdb,'.tbl_SRCHSStatusHistory
SELECT cast(RowIdentifier as int64) as RowIdentifier, 
IDOrganisationVisibleTo, 
IDEvent, 
IDPatient, 
Digest, 
Cast(left(DateEventRecorded,19) as date) as DateEventRecorded,
Cast(left(DateEvent,19)  as date) as DateEvent,
IDProfileEnteredBy, 
IDDoneBy, 
IDOrganisationDoneAt, 
TextualEventDoneBy, 
CHSStatusGroup, 
CHSStatus, 
IDOrganisation, 
IDOrganisationRegisteredAt, 
RemovedData, 
cb_loadnum, 
cb_exported_to_sde  ,
cast(cb_rowid as int64) as cb_rowid ,
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SRCHSStatusHistory', sep = "")

tb5 <- bq_project_query(project_id, sql5)

eric <- paste("tbl_SRCHStatusHistory done "  ,Sys.time() + hours(1) ,sep = "")
print(eric) 



# tbl_SRCode

sql6 <-paste('delete from ', sourcedb,'.tbl_SRCode
where Digest is null;', sep = "")

tb6 <- bq_project_query(project_id, sql6)



sql7 <-paste('insert into   ',targetdb,'.tbl_SRCode
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
demo.person_id ,
IDOrganisationVisibleTo,
Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
Cast(left(DateEvent,19)  as datetime) as DateEvent,
cast(IDProfileEnteredBy as int64) as IDProfileEnteredBy,
cast(IDDoneBy as int64) as IDDoneBy ,
TextualEventDoneBy,
IDOrganisationDoneAt,
CTV3Code,
CTV3Text,
SNOMEDCode,
SNOMEDText,
cast(NumericComparator as int64) ,
IsNumeric,
cast(NumericValue as float64) ,
NumericUnit,
cast(EpisodeType as int64),
cast(IDTemplate as int64),
cast(IDEvent as int64),
cast(IDPatient as int64),
cod.Digest,
cast(IDReferralIN as int64),
cast(IDAppointment as int64),
cast(IDVisit as int64),
IDOrganisation,
IDOrganisationRegisteredAt,
RemovedData, 
cb_loadnum,
cb_exported_to_sde ,
cast(cb_rowid as int64) as cb_rowid ,
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SRCode cod
left join ', demo, '  demo
on cod.Digest = demo.digest ;', sep = "")

tb7 <- bq_project_query(project_id, sql7)


sql8 <-paste('update  ',targetdb,'.tbl_SRCode src
set src.SNOMEDCode = sno.SNOMEDCode
 from yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_CTV3ToSnomed_Map sno
 where src.CTV3Code = sno.CTV3Code
 and src.SNOMEDCode is null;', sep = "")

tb8 <- bq_project_query(project_id, sql8)
 
eric <- paste("tbl_SRCode done "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 


#tbl_SRCodeTemplateLink

sql9 <-paste('delete from ', sourcedb,'.tbl_SRCodeTemplateLink 
where Digest is null;', sep = "")

tb9 <- bq_project_query(project_id, sql9)


sql10 <-paste('insert into   ',targetdb,'.tbl_SRCodeTemplateLink
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
demo.person_id ,
IDPatient,
IDOrganisationVisibleTo,
IDEvent,
IDCode,
IDTemplateRecording,
IDTemplate,
DateEventRecorded,
cod.digest,
RemovedData, 
cb_loadnum,
cb_exported_to_sde ,
cast(cb_rowid as int64) ,
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SRCodeTemplateLink cod
left join ', demo, '  demo
on cod.Digest = demo.digest;', sep = "")

tb10 <- bq_project_query(project_id, sql10)

eric <- paste("tbl_SRCodeTemplateLink done "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 

#tbl_SRConfiguredListOption

sql11 <-paste('delete From ', sourcedb,'.tbl_SRConfiguredListOption
where ConfiguredListOption is null;', sep = "")
tb11 <- bq_project_query(project_id, sql11)

sql12 <-paste('delete From ', sourcedb,'.tbl_SRConfiguredListOption
where RowIdentifier like "%---%";', sep = "")

tb12 <- bq_project_query(project_id, sql12)

sql13 <-paste('insert into  ',targetdb,'.tbl_SRConfiguredListOption 
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier,
 ConfiguredList,
ConfiguredListOption,
CDSCode,
CAMHSCode,
MHSDSCode,
RemovedData, 
cb_loadnum,
cb_exported_to_sde ,
cast(cb_rowid as int64) as cb_rowid,
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
 From ', sourcedb,'.tbl_SRConfiguredListOption ;', sep = "")
tb13 <- bq_project_query(project_id, sql13)
 
eric <- paste("tbl_SRConfiguredListOption done "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 





#tbl_DrugSensitivity
sql14 <-paste('delete from ', sourcedb,'.tbl_SRDrugSensitivity
where Digest is null;', sep = "")
tb14 <- bq_project_query(project_id, sql14)

sql15 <-paste('delete from ', sourcedb,'.tbl_SRDrugSensitivity
where IDOrganisationVisibleTo is null;', sep = "")
tb15 <- bq_project_query(project_id, sql15)

#delete from  ',targetdb,'.tbl_SRDrugSensitivity
#where RowIdentifier in (SELECT distinct cast(RowIdentifier as int64)  from   ', sourcedb,'.tbl_SRDrugSensitivity);
sql16 <-paste('insert into  ',targetdb,'.tbl_SRDrugSensitivity
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
demo.person_id ,
IDOrganisationVisibleTo,
Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
Cast(left(DateEvent,19)  as datetime) as DateEvent,
IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
Cast(left(DateStarted,19) as datetime) as DateStarted,
Cast(left(DateEnded,19) as datetime) as DateEnded,
FormulationSpecific,
IDDrugCode,
IDMultiLexAction,
IDReferralIn,
IDEvent,
IDPatient,
drg.Digest,
IDOrganisation,
IDOrganisationRegisteredAt,
RemovedData, 
cb_loadnum,
cb_exported_to_sde,
cast(cb_rowid as int64) ,
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
 From ', sourcedb,'.tbl_SRDrugSensitivity drg
left join ', demo, '  demo
on drg.Digest = demo.digest;', sep = "")
tb16 <- bq_project_query(project_id, sql16)

 
eric <- paste("tbl_SRDrugSensitivity done "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 

#tbl_SREvent
sql17 <-paste('delete from ', sourcedb,'.tbl_SREvent
where Digest is null;', sep = "")
tb17 <- bq_project_query(project_id, sql17)

sql18 <-paste('delete from ', sourcedb,'.tbl_SREvent
where IDOrganisationVisibleTo is null;', sep = "")
tb18 <- bq_project_query(project_id, sql18)
#delete from  ',targetdb,'.tbl_SREvent
#where RowIdentifier in (SELECT distinct cast(RowIdentifier as int64)  from   ', sourcedb,'.tbl_SREvent);

sql19 <-paste('insert into  ',targetdb,'.tbl_SREvent
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
demo.person_id ,
IDOrganisationVisibleTo,
Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
Cast(left(DateEvent,19)  as datetime) as DateEvent,
IDProfileEnteredBy,
IDDoneBy,
IDStaffMemberProfileRole,
TextualEventDoneBy,
IDAuthorisedBy,
IDProfileAuthorisedBy,
ContactEventLocation,
ContactMethod,
EventIncomplete,
ClinicalEvent,
IDReferralin,
IDPatient,
Sre.Digest,
IDOrganisation,
IDTeam,
IDBranch,
IDOrganisationRegisteredAt,
RemovedData, 
IDOrganisationDoneAt,
cb_exported_to_sde,
cb_loadnum ,
cast(cb_rowid as int64) ,
DuplicateData ,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SREvent sre 
left join ', demo, '  demo
on sre.Digest = demo.digest;', sep = "")
tb19 <- bq_project_query(project_id, sql19)

eric <- paste("tbl_SREvent done "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 

In [4]:
#tbl_SREventLink
sql20 <-paste('delete from ', sourcedb,'.tbl_SREventLink
where Digest is null;', sep = "")

tb20 <- bq_project_query(project_id, sql20)

sql21 <-paste('delete from ', sourcedb,'.tbl_SREventLink
where IDOrganisationVisibleTo is null;', sep = "")

tb21 <- bq_project_query(project_id, sql21)

#delete from  ',targetdb,'.tbl_SREventLink
#where RowIdentifier in (SELECT distinct cast(RowIdentifier as int64)  from   ', sourcedb,'.tbl_SREventLink);
sql22 <-paste('insert into  ',targetdb,'.tbl_SREventLink 
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier
,demo.person_id 
,IDOrganisationVisibleTo
,Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded
,Cast(left(DateEvent,19)  as datetime) as DateEvent
,IDProfileEnteredBy
,IDDoneBy
,TextualEventDoneBy
,IDOrganisationDoneAt
,IDEvent
,IDPatient
,sre.Digest
,IDReferralin
,IDOrganisation
,IDAppointment
,IDVisit
,IDOrganisationRegisteredAt
,RemovedData 
,cb_loadnum
,cb_exported_to_sde 
,cast(cb_rowid as int64) 
,cb_duplicate_data
,cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SREventLink sre
left join ', demo, '  demo
on sre.Digest = demo.digest;', sep = "")

tb22 <- bq_project_query(project_id, sql22)

eric <- paste("tbl_SREventLink done "  ,Sys.time() + hours(1) ,sep = "")
print(eric) 

[1] "tbl_SREventLink done 2024-02-08 14:58:24.405691"


In [5]:
#tbl_SRGPPracticeHistory
sql23 <-paste('delete from ', sourcedb,'.tbl_SRGPPracticeHistory
where Digest is null;', sep = "")
tb23 <- bq_project_query(project_id, sql23)

sql24 <-paste('delete from ', sourcedb,'.tbl_SRGPPracticeHistory
where IDOrganisationVisibleTo is null;', sep = "")
tb24 <- bq_project_query(project_id, sql24)




#delete from  ',targetdb,'.tbl_SRGPPracticeHistory
#where RowIdentifier in (SELECT distinct cast(RowIdentifier as int64)  from   ', sourcedb,'.tbl_SRGPPracticeHistory);
sql25 <-paste('insert into  ',targetdb,'.tbl_SRGPPracticeHistory 
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier,
IDOrganisationVisibleTo,
IDPractice,
IDProfileRegisteredGP,
cast( DateFrom as Datetime) as DateFrom ,
cast(DateTo as Datetime) as  DateTo,
IDPatient,
prac.Digest,
RemovedData, 
cast(cb_loadnum as int64) as cb_loadnum,
cb_exported_to_sde ,
cast(cb_rowid as int64) ,
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SRGPPracticeHistory prac
left join ', demo, '  demo
on prac.Digest = demo.digest;', sep = "")
tb25 <- bq_project_query(project_id, sql25)

eric <- paste("tbl_SRGPPracticeHistory done "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 


 

#tbl_SRGoal
 
sql26 <-paste('delete from ', sourcedb,'.tbl_SRGoal
where Digest is null ;', sep = "")
tb26 <- bq_project_query(project_id, sql26)


sql27 <-paste('delete from ', sourcedb,'.tbl_SRGoal
where IDOrganisationVisibleTo is null;', sep = "")
tb27 <- bq_project_query(project_id, sql27)



sql28 <-paste('insert into  ',targetdb,'.tbl_SRGoal 
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
demo.person_id ,
 IDOrganisationVisibleTo,
Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
Cast(left(DateEvent,19)  as datetime) as DateEvent,
IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
GoalCategory,
GoalType,
GoalOutcome,
GoalImportance,
DateGoalTarget,
ActionPlanCategory,
ActionPlanPatientConfidence,
ActionPlanOutcome,
DateEndedActionPlan,
IDEvent,
IDPatient,
IDOrganisation,
IDOrganisationRegisteredAt,
GoalPerformance,
Gol.digest,
RemovedData, 
cb_loadnum,
cb_exported_to_sde ,
cast(cb_rowid as int64),
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SRGoal gol
      left join ', demo, '  demo
on gol.Digest = demo.digest;', sep = "")
tb28 <- bq_project_query(project_id, sql28)

eric <- paste("tbl_SRGoal done "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 




#tbl_SRImmunisation
sql29 <-paste('delete from ', sourcedb,'.tbl_SRImmunisation
where Digest is null;', sep = "")
tb28 <- bq_project_query(project_id, sql29)


sql30 <-paste('delete from ', sourcedb,'.tbl_SRImmunisation
where IDOrganisationVisibleTo is null;', sep = "")
tb30 <- bq_project_query(project_id, sql30)



sql31 <-paste('insert into  ',targetdb,'.tbl_SRImmunisation 
SELECT distinct  cast(RowIdentifier as int64) as RowIdentifier ,
demo.person_id ,
 IDOrganisationVisibleTo,
Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
Cast(left(DateEvent,19)  as datetime) as DateEvent,
IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
IDVaccination,
IDImmunisationContent,
Dose,
Location,
Method,
DateExpiry,
ImmsReadCode,
VaccPart,
VaccBatchNumber,
VaccAreaCode,
VaccinationStatus,
IDEvent,
IDPatient,
imm.Digest,
IDOrganisation,
IDOrganisationRegisteredAt,
RemovedData, 
cb_loadnum,
cb_exported_to_sde ,
cast(cb_rowid as int64) ,
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SRImmunisation imm
  left join ', demo, '  demo
on imm.Digest = demo.digest;', sep = "")
tb31 <- bq_project_query(project_id, sql31)

eric <- paste("tbl_SRImmunisation done "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 
 



#tbl_SRImmunisationConsent

sql32 <-paste('delete from ', sourcedb,'.tbl_SRImmunisationConsent
where Digest is null;', sep = "")
tb32 <- bq_project_query(project_id, sql32)


sql33 <-paste('delete from ', sourcedb,'.tbl_SRImmunisationConsent
where IDOrganisationVisibleTo is null;', sep = "")
tb33 <- bq_project_query(project_id, sql33)



sql34 <-paste('insert into  ',targetdb,'.tbl_SRImmunisationConsent 
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
demo.person_id ,
 IDOrganisationVisibleTo,
Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
Cast(left(DateEvent,19)  as datetime) as DateEvent,
IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
ConsentedToVacc,
RefusalOrConsentCurrent,
DateEnded,
VaccName,
VaccContent,
IDEvent,
IDPatient,
Imm.Digest,
IDOrganisation,
IDOrganisationRegisteredAt,
RemovedData, 
cb_loadnum,
cb_exported_to_sde ,
cast(cb_rowid as int64) ,
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SRImmunisationConsent imm
  left join ', demo, '  demo
on imm.Digest = demo.digest;', sep = "")
tb34 <- bq_project_query(project_id, sql34)

eric <- paste("tbl_SRImmunisationConsent done "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 
 

#tbl_SRImmunisationContent
sql35 <-paste('delete from   ', sourcedb,'.tbl_SRImmunisationContent
where Name is null ;', sep = "")
tb35 <- bq_project_query(project_id, sql35)




sql36 <-paste('insert into  ',targetdb,'.tbl_SRImmunisationContent 
SELECT distinct cast(RowIdentifier as int64) 
, Name
, Content
, DateDeleted
, RemovedData
, cast(cb_loadnum as int64) 
, cb_exported_to_sde ,
cast(cb_rowid as int64) ,
DuplicateData ,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SRImmunisationContent ;', sep = "")
tb36 <- bq_project_query(project_id, sql36)

eric <- paste("tbl_SRImmunisationContent done "  ,Sys.time() + hours(1) ,sep = "")
print(eric) 

#tbl_Mapping
sql37 <-paste('delete from   ', sourcedb,'.tbl_SRMapping
where Mapping is null ;', sep = "")
tb37 <- bq_project_query(project_id, sql37)



sql38 <-paste('insert into  ',targetdb,'.tbl_SRMapping 
SELECT
cast(RowIdentifier as int64) as RowIdentifier
, IdMappingGroup
, Mapping 
,RemovedData
, cb_loadnum
, cb_exported_to_sde,
cast(cb_rowid as int64) ,
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
 From ', sourcedb,'.tbl_SRMapping;', sep = "")
tb38 <- bq_project_query(project_id, sql38)

eric <- paste("tbl_SRMappingdone "  ,Sys.time() + hours(1) ,sep = "")
print(eric) 

#tbl_MappingGroup
sql39 <-paste('delete from   ', sourcedb,'.tbl_SRMappingGroup
where GroupName is null ;', sep = "")
tb39 <- bq_project_query(project_id, sql39)



sql40 <-paste('insert into  ',targetdb,'.tbl_SRMappingGroup 
SELECT
cast(RowIdentifier as int64) as RowIdentifier
, GroupName
,RemovedData
, cb_loadnum
, cb_exported_to_sde,
cast(cb_rowid as int64) ,
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
 From ', sourcedb,'.tbl_SRMappingGroup;', sep = "")
tb40 <- bq_project_query(project_id, sql40)

eric <- paste("tbl_SRMappingGroup  done "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 

#tbl_SRMedicationReadCodeDetails 
sql41 <-paste('delete from   ', sourcedb,'.tbl_SRMedicationReadCodeDetails
where IDMultiLexProduct is null ;', sep = "")
tb41 <- bq_project_query(project_id, sql41)


#delete from  ',targetdb,'.tbl_SRMedicationReadCodeDetails
#where RowIdentifier in (SELECT distinct cast(RowIdentifier as int64)  from   ', sourcedb,'.tbl_SRMedicationReadCodeDetails);

sql42 <-paste('insert into  ',targetdb,'.tbl_SRMedicationReadCodeDetails 
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier 
, IDMultiLexProduct
, DrugReadCode
, DrugReadCodeDesc
,RemovedData
, cb_loadnum
, cb_exported_to_sde ,
cast(cb_rowid as int64) ,
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SRMedicationReadCodeDetails ;', sep = "")
tb42 <- bq_project_query(project_id, sql42)

eric <- paste("tbl_SRMedicationReadCodeDetails done "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 
 
 

#tbl_SROnlineServices
sql43 <-paste('delete from   ', sourcedb,'.tbl_SROnlineServices 
where Digest is null ;', sep = "")
tb43 <- bq_project_query(project_id, sql43)



#delete from  ',targetdb,'.tbl_SROnlineServices 
#where RowIdentifier in (SELECT distinct cast(RowIdentifier as int64)  from   ', sourcedb,'.tbl_SROnlineServices);
sql44 <-paste('insert into  ',targetdb,'.tbl_SROnlineServices
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 IDEvent, 
 IDPatient,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded
, cast(left(DateEvent,19) as datetime) as DateEvent
, IDProfileEnteredBy, 
IDDoneBy,
 TextualEventDoneBy, 
 IDOrganisation, 
 IDOrganisationDoneAt, 
 ViewRecord, 
 Appointments, 
 Prescriptions, 
 Questionnaires, 
 SummaryCareRecord, 
 DateDeleted, 
 IDProfileDeletedBy, 
 DetailedCodedRecord, 
 IDOnlineUser, 
 IDOrganisationRegisteredAt, 
 sv.digest, 
RemovedData,
 cb_loadnum, 
cb_exported_to_sde,
cast(cb_rowid as int64) ,
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SROnlineServices sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;', sep = "")
tb44 <- bq_project_query(project_id, sql44)

eric <- paste("tbl_SROnlineServices done "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 

#tbl_SROrganisation 
sql45 <-paste('delete from ', sourcedb,'.tbl_SROrganisation where Name is null ;', sep = "")
tb45 <- bq_project_query(project_id, sql45)

sql45a <-paste('update ', sourcedb,'.tbl_SROrganisation set cb_record_added = null where cb_record_added = "NULL" ;', sep = "")
tb45a <- bq_project_query(project_id, sql45a)

sql45b <-paste('update ', sourcedb,'.tbl_SROrganisation set cb_exported_to_sde = null where cb_exported_to_sde = "NULL" ;', sep = "")
tb45b <- bq_project_query(project_id, sql45b)

sql46 <-paste('insert into  ',targetdb,'.tbl_SROrganisation
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
Name, 
 ID, 
 HouseName, 
 HouseNumber, 
 NameOfRoad, 
 NameOfLocality, 
 NameOfTown,
  NameOfCounty, 
  FullPostCode, 
  Telephone, 
  SecondaryTelephone,
   Fax, 
  MadeObsolete, 
  IDTrust, 
  IDCcg, 
RemovedData,
cast(cb_loadnum as int64) as cb_loadnum,
cast(cb_exported_to_sde as Timestamp) as cb_exported_to_sde,
cast(cb_rowid as int64) ,
DuplicateData ,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SROrganisation  sv;', sep = "")
tb46 <- bq_project_query(project_id, sql46)

eric <- paste("tbl_SROrganisation done "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 

 
#tbl_SROrganisationBranch
sql47 <-paste('delete from   ', sourcedb,'.tbl_SROrganisationBranch where BranchName is null ;', sep = "")
tb47 <- bq_project_query(project_id, sql47)



sql48 <-paste('insert into  ',targetdb,'.tbl_SROrganisationBranch
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier, 
IDOrganisationVisibleTo, 
ID, 
BranchName, 
HouseName, 
HouseNumber, 
RoadName, 
Locality, 
Town, 
County, 
PostCode, 
BranchObsolete, 
IDOrganisation, 
RemovedData,
cb_loadnum, 
cb_exported_to_sde ,
cast(cb_rowid as int64),
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SROrganisationBranch ;', sep = "")
tb48 <- bq_project_query(project_id, sql48)

eric <- paste("tbl_SROrganisationBranch done "  ,Sys.time() + hours(1) ,sep = "")
print(eric) 
 

#tbl_SROverseasVisitorChargingCategory
sql49 <-paste('delete from   ', sourcedb,'.tbl_SROverseasVisitorChargingCategory where Digest is null ;', sep = "")
tb49 <- bq_project_query(project_id, sql49)




sql50 <-paste('insert into  ',targetdb,'.tbl_SROverseasVisitorChargingCategory
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 IDEvent, 
 OverseasVisitorChargingCategory, 
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
 Cast(left(DateEvent,19) as datetime) as DateEvent,
IDProfileEnteredBy, 
IDDoneBy, 
TextualEventDoneBy, 
IDOrganisationDoneAt, 
IDOrganisation, 
IDPatient, 
IDReferralIn, 
RemovedData,
sv.digest, 
cb_loadnum, 
cb_exported_to_sde ,
cb_rowid ,
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SROverseasVisitorChargingCategory sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;', sep = "")
tb50 <- bq_project_query(project_id, sql50)

eric <- paste("tbl_SROverseasVisitorChargingCategory done "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 

#tbl_SRPatient
sql51 <-paste('delete from   ', sourcedb,'.tbl_SRPatient where Digest is null ;', sep = "")
tb51 <- bq_project_query(project_id, sql51)



sql51 <-paste('insert into  ',targetdb,'.tbl_SRPatient
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 sv.Digest, 
 DateBirth, 
 DateDeath, 
 BirthPlace, 
 Gender, 
 SpeaksEnglish, 
 TestPatient, 
 SSRef, 
 SpineMatched, 
 Ethnicity, 
 LanguageSpoken, 
 SOA, 
 sv.Ward, 
 Religion, 
RemovedData,
 cb_loadnum, 
 cb_exported_to_sde ,
cb_rowid ,
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
 From ', sourcedb,'.tbl_SRPatient  sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;', sep = "")
tb51 <- bq_project_query(project_id, sql51)

eric <- paste("tbl_SRPatient done "  ,Sys.time() + hours(1) ,sep = "")
print(eric) 

#tbl_SRPatientAddressHistory
sql52 <-paste('delete from   ', sourcedb,'.tbl_SRPatientAddressHistory where Digest is null ;', sep = "")
tb52 <- bq_project_query(project_id, sql52)



sql53 <-paste('insert into  ',targetdb,'.tbl_SRPatientAddressHistory
SELECT distinct cast(RowIdentifier as int64) ,
person_id,
IDOrganisationVisibleTo,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
sv.PartialPostCode,
DateTo,
AddressType,
IDEvent,
IDPatient,
sv.Digest,
IDOrganisation,
CcgOfResidence,
IDOrganisationRegisteredAt,
RemovedData,
cb_loadnum,
cb_exported_to_sde ,
cb_rowid ,
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SRPatientAddressHistory sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;', sep = "")
tb53 <- bq_project_query(project_id, sql53)

eric <- paste("tbl_SRPatientAddressHistory done "  ,Sys.time() + hours(1) ,sep = "")
print(eric) 

#tbl_SRPatientGroups
sql54 <-paste('delete from   ', sourcedb,'.tbl_SRPatientGroups where Digest is null ;', sep = "")
tb54 <- bq_project_query(project_id, sql54)



sql55 <-paste('insert into  ',targetdb,'.tbl_SRPatientGroups
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
 Cast(left(DateEvent,19) as datetime) as DateEvent,
 IDProfileEnteredBy, 
Name, 
GroupType, 
Members, 
DateRemoved, 
IDStaffRemoved, 
IDProfileRemoved, 
IDEvent, 
IDPatient, 
IDOrganisation, 
IDOrganisationRegisteredAt, 
sv.digest, 
RemovedData,
cb_loadnum, 
cb_exported_to_sde ,
cb_rowid ,
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
 From ', sourcedb,'.tbl_SRPatientGroups sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;', sep = "")
tb55 <- bq_project_query(project_id, sql55)

eric <- paste("tbl_SRPatientGroups done "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 

#tbl_SRPatientRegistration
sql56 <-paste('delete from   ', sourcedb,'.tbl_SRPatientRegistration where Digest is null ;', sep = "")
tb56 <- bq_project_query(project_id, sql56)

sql56a <-paste('update   ', sourcedb,'.tbl_SRPatientRegistration set cb_record_added = null where cb_record_added = "NULL" ;', sep = "")
tb56a <- bq_project_query(project_id, sql56a)

sql56a <-paste('update   ', sourcedb,'.tbl_SRPatientRegistration set cb_exported_to_sde = null where cb_exported_to_sde = "NULL" ;', sep = "")
tb56a <- bq_project_query(project_id, sql56a)



sql57 <-paste('insert into  ',targetdb,'.tbl_SRPatientRegistration
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
 #Cast(left(DateEvent,19) as datetime) as DateEvent,
IDProfileEnteredBy, 
DateRegistration, 
DateDeRegistration, 
RegistrationStatus, 
PreferredPharmacy,
 IDPatient, 
 sv.Digest, 
 IDOrganisation, 
 IDOrganisationRegisteredAt, 
RemovedData,
 cb_loadnum, 
 cb_exported_to_sde ,
cb_rowid ,
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
 From ', sourcedb,'.tbl_SRPatientRegistration  sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;', sep = "")
tb57 <- bq_project_query(project_id, sql57)

eric <- paste("tbl_SRPatientRegistration done "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 



#tbl_SRPatientRelationship
sql58 <-paste('delete from   ', sourcedb,'.tbl_SRPatientRelationship where Digest is null ;', sep = "")
tb58 <- bq_project_query(project_id, sql58)



sql59 <-paste('insert into  ',targetdb,'.tbl_SRPatientRelationship
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
 Cast(left(DateEvent,19) as datetime) as DateEvent,
IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
DateEnded,
RelationshipType,
PersonalGuardianOrProxy,
NextOfKin,
CaresForPatient,
PrincipalCarerForPatient,
KeyHolder,
HasParentalResponsibility,
FinancialRepresentative,
Advocate,
MainVisitor,
CallCentreCallBackConsent,
CopyCorrespondence,
ContactOrder,
ContactMethod,
CommunicationFormat,
InterpreterRequired,
IDRelationshipWithPatient,
IDPatientRelationshipWith,
CodeRelationshipWithUser,
RelationshipWithOrganisation,
IDEvent,
IDPatient,
IDOrganisation,
IDOrganisationRegisteredAt,
sv.digest,
RemovedData,
cb_loadnum,
cb_exported_to_sde 
,cast(cb_rowid as int64),
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added  
From ', sourcedb,'.tbl_SRPatientRelationship sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;', sep = "")
tb59 <- bq_project_query(project_id, sql59)

eric <- paste("tbl_SRPatientRelationship done "  ,Sys.time() + hours(1) ,sep = "")
print(eric) 

#tbl_SRPersonAtRisk
sql60 <-paste('delete from   ', sourcedb,'.tbl_SRPersonAtRisk where Digest is null ;', sep = "")
tb60 <- bq_project_query(project_id, sql60)



sql60 <-paste('insert into  ',targetdb,'.tbl_SRPersonAtRisk
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
DateAdded,
IDProfileEnteredBy,
ProtectionPlan,
DateRemoved,
IDProfileRemovedBy,
IDPatient,
IDOrganisation,
IDOrganisationRegisteredAt,
sv.digest,
RemovedData,
cb_loadnum,
cb_exported_to_sde ,
cast(cb_rowid as int64) ,
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SRPersonAtRisk sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;', sep = "")
tb60 <- bq_project_query(project_id, sql60)

eric <- paste("tbl_SRPersonAtRisk done "  ,Sys.time() + hours(1) ,sep = "")
print(eric) 

#tbl_SRPrimaryCareMedication 
sql61 <-paste('delete from   ', sourcedb,'.tbl_SRPrimaryCareMedication 
where Digest is null ;', sep = "")
tb61 <- bq_project_query(project_id, sql61)



sql62 <-paste('insert into  ',targetdb,'.tbl_SRPrimaryCareMedication 
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
Cast(left(DateEvent,19)  as datetime) as DateEvent,
 IDProfileEnteredBy,
 IDDoneBy,
 TextualEventDoneBy,
 IDOrganisationDoneAt,
 IDMultiLexProduct,
 IDMultiLexPack,
 IDMultiLexDMD,
 NameOfMedication,
Cast(left(DateMedicationStart,19) as datetime) as DateMedicationStart,
Cast(left(DateMedicationEnd,19)  as datetime) as DateMedicationEnd,
 MedicationDosage,
 MedicationQuantity,
 IsSupervised,
 SupervisedDose,
 UnSupervisedDose,
 IsRepeatMedication,
 IsOtherMedication,
 IsDentalMedication,
 IsHospitalMedication,
 IDRepeatTemplate,
 IDReferralIN,
 IDEvent,
 IDPatient,
 med.Digest,
 IDOrganisation,
 IDOrganisationRegisteredAt,
RemovedData,
 cb_loadnum,
 cb_exported_to_sde ,
cast(cb_rowid as int64) ,
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
 From ', sourcedb,'.tbl_SRPrimaryCareMedication med 
  left join ', demo, '  demo
on med.Digest = demo.digest;', sep = "")
tb62 <- bq_project_query(project_id, sql62)

eric <- paste("tbl_SRPrimaryCareMedication  done "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 


#tbl_SRProblem
sql63 <-paste('delete from   ', sourcedb,'.tbl_SRProblem where Digest is null ;', sep = "")
tb63 <- bq_project_query(project_id, sql63)



sql64 <-paste('insert into  ',targetdb,'.tbl_SRProblem
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
 Cast(left(DateEvent,19) as datetime) as DateEvent,
IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
CTV3Code,
DateEnd,
Severity,
IDCode,
IDEvent,
IDPatient,
sv.Digest,
IDReferralIN,
IDOrganisation,
IDOrganisationRegisteredAt,
RemovedData,
cb_loadnum,
cb_exported_to_sde ,
cast(cb_rowid as int64),
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SRProblem  sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;', sep = "")
tb64 <- bq_project_query(project_id, sql64)

eric <- paste("tbl_SRProblem done "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 

#tbl_SRRecall
sql65 <-paste('delete from   ', sourcedb,'.tbl_SRRecall where Digest is null ;', sep = "")
tb65 <- bq_project_query(project_id, sql65)



sql66 <-paste('insert into  ',targetdb,'.tbl_SRRecall
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
 Cast(left(DateEvent,19) as datetime) as DateEvent,
  IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
DateRecall,
RecallType,
RecallInitialStatus,
RecallStatus,
RecallStatusDate,
IDReferralIN,
IDEvent,
IDPatient,
sv.Digest,
IDOrganisation,
IDOrganisationRegisteredAt,
RemovedData,
cb_loadnum,
cb_exported_to_sde ,
cast(cb_rowid as int64),
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SRRecall sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;', sep = "")
tb66 <- bq_project_query(project_id, sql66)

eric <- paste("tbl_SRRecall done "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 

#tbl_SRReferralIn
sql67 <-paste('delete from   ', sourcedb,'.tbl_SRReferralIn where Digest is null ;', sep = "")
tb67 <- bq_project_query(project_id, sql67)



sql68 <-paste('insert into  ',targetdb,'.tbl_SRReferralIn
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
 Cast(left(DateEvent,19) as datetime) as DateEvent,
 IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
Category,
PrimaryReason,
DateReferral,
DateOnReferralLetter,
Source,
ServiceOffered,
ServiceType,
ReReferral,
Urgency,
IDCodePrimaryDiagnosis,
ReferralCode,
IDReferralLocal,
ReasonForServiceDelay,
Outcome,
DefaultContactLocation,
DateDischargeScheduled,
DateDischarge,
DischargeReason,
DischargeLocation,
IDReferrer,
SourceTableReferrer,
PatientAware,
DateOfAction,
ActiveStatus,
ReferredToSpeciality,
ReferredToTreatmentCode,
IDReferredTo,
SourceTableReferrerTo,
ReferralPriority,
ReferralMedium,
DateOfDecision,
DayCareReferral,
MaternityReferral,
Commissioner,
ClientCategory,
PrimarySupportReason,
IDCaseload,
IDEvent,
IDPatient,
sv.Digest,
IDOrganisation,
ReferralEndReason,
HospitalReferralSourceCdsCode,
M101140ReasonForOutOfAreaReferralAdultAcuteMentalHealth,
IDOrganisationRegisteredAt,
HospitalReferredForType,
RemovedData,
cb_loadnum,
cb_exported_to_sde ,
cb_rowid,
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added 
From ', sourcedb,'.tbl_SRReferralIn sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;', sep = "")
tb68 <- bq_project_query(project_id, sql68)

eric <- paste("tbl_SRReferralIn done "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 

#tbl_SRReferralInIntervention
sql69 <-paste('delete from   ', sourcedb,'.tbl_SRReferralInIntervention where Digest is null ;', sep = "")
tb69 <- bq_project_query(project_id, sql69)




sql70 <-paste('insert into  ',targetdb,'.tbl_SRReferralInIntervention
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
 Cast(left(DateEvent,19) as datetime) as DateEvent,
  IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
Intervention,
IDReferralIN,
IDEvent,
IDPatient,
sv.Digest,
IDOrganisation,
IDOrganisationRegisteredAt,
RemovedData,
cb_loadnum,
cb_exported_to_sde ,
cast(cb_rowid as int64),
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added 
 From ', sourcedb,'.tbl_SRReferralInIntervention sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;', sep = "")
tb70 <- bq_project_query(project_id, sql70)

eric <- paste("tbl_SRReferralInIntervention done "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 

#tbl_SRReferralInReferralReason
sql71 <-paste('delete from   ', sourcedb,'.tbl_SRReferralInReferralReason where Digest is null ;', sep = "")
tb71 <- bq_project_query(project_id, sql71)



sql72 <-paste('insert into  ',targetdb,'.tbl_SRReferralInReferralReason
 SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
 Cast(left(DateEvent,19) as datetime) as DateEvent,
IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
PrimaryReferralReason,
ReferralReason,
DateRemoved,
IDReferralIN,
IDEvent,
IDPatient,
sv.Digest,
IDOrganisation,
IDOrganisationRegisteredAt,
RemovedData,
cb_loadnum,
cb_exported_to_sde ,
cast(cb_rowid as int64),
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SRReferralInReferralReason sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;', sep = "")
tb72 <- bq_project_query(project_id, sql72)

eric <- paste("tbl_SRReferralInReferralReason done "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 

#tbl_SRReferralInStatusDetails
sql73 <-paste('delete from   ', sourcedb,'.tbl_SRReferralInStatusDetails where Digest is null ;', sep = "")
tb73 <- bq_project_query(project_id, sql73)



sql74 <-paste('insert into  ',targetdb,'.tbl_SRReferralInStatusDetails
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
 Cast(left(DateEvent,19) as datetime) as DateEvent,
 IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
StatusOfReferralIN,
IDReferralIn,
IDEvent,
IDPatient,
sv.Digest,
IDOrganisation,
IDOrganisationRegisteredAt,
RemovedData,
cb_loadnum,
cb_exported_to_sde ,
cast(cb_rowid as int64) ,
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SRReferralInStatusDetails sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;', sep = "")
tb74 <- bq_project_query(project_id, sql74)

eric <- paste("tbl_SRReferralInStatusDetails done "  ,Sys.time() + hours(1) ,sep = "")
print(eric) 

#tbl_SRReferralOut
sql75 <-paste('delete from   ', sourcedb,'.tbl_SRReferralOut where Digest is null ;', sep = "")
tb75 <- bq_project_query(project_id, sql75)



sql76 <-paste('insert into  ',targetdb,'.tbl_SRReferralOut
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
 Cast(left(DateEvent,19) as datetime) as DateEvent,
 IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
TypeOfReferral,
Reason,
IDProfileReferrer,
ServiceOffered,
ReReferral,
Urgency,
PrimaryDiagnosis,
RecipientID,
RecipientIDType,
IDEvent,
IDPatient,
sv.Digest,
IDOrganisation,
M101140ReasonForOutOfAreaReferralAdultAcuteMentalHealth,
IDOrganisationRegisteredAt,
RemovedData,
cb_loadnum,
cb_exported_to_sde ,
cast(cb_rowid as int64) ,
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SRReferralOut sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;', sep = "")

tb76 <- bq_project_query(project_id, sql76)

eric <- paste("tbl_SRReferralOut done "  ,Sys.time() + hours(1) ,sep = "")
print(eric) 

#tbl_SRReferralOutStatusDetails
sql77 <-paste('delete from   ', sourcedb,'.tbl_SRReferralOutStatusDetails where Digest is null ;', sep = "")
tb77 <- bq_project_query(project_id, sql77)



sql78 <-paste('insert into  ',targetdb,'.tbl_SRReferralOutStatusDetails
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
 Cast(left(DateEvent,19) as datetime) as DateEvent,
IDOrganisationDoneAt,
IDProfileEnteredBy,
TextualEventDoneBy,
IDDoneBy,
StatusOfReferralOut,
IDReferralOut,
IDEvent,
IDPatient,
sv.Digest,
IDOrganisation,
IDOrganisationRegisteredAt,
RemovedData,
cb_loadnum,
cb_exported_to_sde ,
cast(cb_rowid as int64),
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SRReferralOutStatusDetails  sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;', sep = "")
tb78 <- bq_project_query(project_id, sql78)

eric <- paste("tbl_SRReferralOutStatusDetails done "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 

#tbl_SRRepeatTemplate
sql79 <-paste('delete from   ', sourcedb,'.tbl_SRRepeatTemplate where Digest is null ;', sep = "")
tb79 <- bq_project_query(project_id, sql79)



sql80 <-paste('insert into  ',targetdb,'.tbl_SRRepeatTemplate
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
 Cast(left(DateEvent,19) as datetime) as DateEvent,
 IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
IDMultiLexProduct,
IDMultiLexPack,
IDMultiLexDMD,
NameOfMedication,
DateMedicationTemplateStart,
DateMedicationTemplateEnd,
DateMedicationTemplateReview,
MedicationDosage,
MedicationQuantity,
MaxIssues,
CourseLengthPerIssue,
DrugStatus,
IDReferralIN,
IDEvent,
IDPatient,
sv.Digest,
IDOrganisation,
IDOrganisationRegisteredAt,
RemovedData,
cb_loadnum,
cb_exported_to_sde ,
cb_rowid ,
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SRRepeatTemplate  sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;', sep = "")
tb80 <- bq_project_query(project_id, sql80)

eric <- paste("tbl_SRRepeatTemplate done "  ,Sys.time() + hours(1) ,sep = "")
print(eric) 

#tbl_SRRota
sql81 <-paste('delete from   ', sourcedb,'.tbl_SRRota where Name is null ;', sep = "")
tb81 <- bq_project_query(project_id, sql81)



sql82 <-paste('insert into  ',targetdb,'.tbl_SRRota
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
IDOrganisationVisibleTo,
 DateCreation,
IDProfileCreatedBy,
Name,
RotaType,
Location,
Code,
IDProfileOwner,
AllowOverbooking,
BookingContactNumber,
IDAppointmentRoom,
IDBranch,
RemovedData,
cb_loadnum,
cb_exported_to_sde ,
cb_rowid ,
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SRRota;', sep = "")
tb82 <- bq_project_query(project_id, sql82)

eric <- paste("tbl_SRRota done "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 

#tbl_SRRotaSlot
sql83 <-paste('delete from   ', sourcedb,'.tbl_SRRotaSlot where IDRota is null ;', sep = "")
tb83 <- bq_project_query(project_id, sql83)



sql84 <-paste('insert into  ',targetdb,'.tbl_SRRotaSlot
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier,
IDOrganisationVisibleTo, 
IDRota, 
RotaSlotType, 
Duration, 
Quantity, 
EmbargoDuration, 
EmbargoExpiryTime, 
BlockedSlot, 
BookableCAnB, 
OldRowIdentifier, 
RemovedData,
cb_loadnum, 
cb_exported_to_sde ,
cast (cb_rowid as int64),
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SRRotaSlot ;', sep = "")
tb84 <- bq_project_query(project_id, sql84)

eric <- paste("tbl_SRRotaSlot done "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 

#tbl_SRSmsConsent

sql85 <-paste('delete from   ', sourcedb,'.tbl_SRSmsConsent where Digest is null ;', sep = "")
tb85 <- bq_project_query(project_id, sql85)



sql86 <-paste('insert into  ',targetdb,'.tbl_SRSmsConsent
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
 Cast(left(DateEvent,19) as datetime) as DateEvent,
 IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
SmsConsent,
IDEvent,
IDPatient,
IDOrganisation,
IDOrganisationRegisteredAt,
sv.digest,
RemovedData,
cb_loadnum,
cb_exported_to_sde ,
cast(cb_rowid as int64),
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SRSmsConsent sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;', sep = "")
tb86 <- bq_project_query(project_id, sql86)

eric <- paste("tbl_SRSmsConsent done "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 

#tbl_SRStaffMember
sql87 <-paste('delete from   ', sourcedb,'.tbl_SRStaffMember where StaffName is null ;', sep = "")
tb87 <- bq_project_query(project_id, sql87)


sql87a <-paste('delete from   ', sourcedb,'.tbl_SRStaffMember where length(cb_record_added) < 10 ;', sep = "")
tb87a <- bq_project_query(project_id, sql87a)


sql88 <-paste('insert into  ',targetdb,'.tbl_SRStaffMember
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
StaffName,
StaffUserName,
NationalIdType,
IDNational,
IDSmartCard,
Obsolete,
RemovedData,
cb_loadnum,
cb_exported_to_sde ,
cast(cb_rowid as int64) as cb_rowid,
DuplicateData ,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SRStaffMember ;', sep = "")
tb88 <- bq_project_query(project_id, sql88)

eric <- paste("tbl_SRStaffMember done "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 

#tbl_SRStaffMemberProfile
sql89 <-paste('delete from   ', sourcedb,'.tbl_SRStaffMemberProfile where StaffRole is null ;', sep = "")
tb89 <- bq_project_query(project_id, sql89)

sql90 <-paste('insert into  ',targetdb,'.tbl_SRStaffMemberProfile
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 DateProfileCreated, 
IDProfileCreatedBy, 
IDStaffMemberProfileRole, 
StaffRole, 
DateEmploymentStart, 
DateEmploymentEnd, 
PPAID, 
GPLocalCode, 
IDStaffMember, 
IDOrganisation, 
GmpID, 
RemovedData,
cb_loadnum, 
cb_exported_to_sde ,
cast(cb_rowid as int64) ,
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SRStaffMemberProfile ;', sep = "")
tb90 <- bq_project_query(project_id, sql90)

eric <- paste("tbl_SRStaffMemberProfile done "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 

#tbl_SRStaffMemberProfileRole 
sql91 <-paste('delete from   ', sourcedb,'.tbl_SRStaffMemberProfileRole where RoleDescription is null ;', sep = "")
tb91 <- bq_project_query(project_id, sql91)



sql92 <-paste('insert into  ',targetdb,'.tbl_SRStaffMemberProfileRole
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
RoleDescription, 
IsSpineRole,
DateLastAmended,
DateDeleted,
RemovedData,
cb_loadnum,
cb_exported_to_sde ,
cast(cb_rowid as int64) ,
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SRStaffMemberProfileRole ;', sep = "")
tb92 <- bq_project_query(project_id, sql92)

eric <- paste("tbl_SRStaffMemberProfileRole done "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 

#tbl_SRTemplate
sql93 <-paste('delete from   ', sourcedb,'.tbl_SRTemplate where TemplateName is null ;', sep = "")
tb93 <- bq_project_query(project_id, sql93)



sql94 <-paste('insert into  ',targetdb,'.tbl_SRTemplate
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
IDOrganisationVisibleTo,
DatePublished,
IDProfilePublishedBy,
TemplateName,
TemplateVersion,
IDOrganisation,
RemovedData,
cb_loadnum,
cb_exported_to_sde ,
cast(cb_rowid as int64),
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SRTemplate;', sep = "")
tb94 <- bq_project_query(project_id, sql94)

eric <- paste("tbl_SRTemplate done "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 

#tbl_SRTrust
sql95 <-paste('delete from   ', sourcedb,'.tbl_SRTrust where Name is null ;', sep = "")
tb95 <- bq_project_query(project_id, sql95)



sql96 <-paste('insert into  ',targetdb,'.tbl_SRTrust
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
Cast(left(DateCreated,19) as datetime) as DateCreated ,
IdProfileCreatedBy,
Name,
OdsCode,
HouseName,
HouseNumber,
NameOfRoad,
NameofLocality,
NameOfTown,
NameOfCounty,
FullPostCode,
Telephone,
SecondaryTelephone,
Fax,
RemovedData,
cb_loadnum,
cb_exported_to_sde ,
cast(cb_rowid as int64) ,
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SRTrust;', sep = "")
tb96 <- bq_project_query(project_id, sql96)

eric <- paste("tbl_SRTrust done "  ,Sys.time() + hours(1) ,sep = "")
print(eric) 

#tbl_SRVariableDoseCDMedication
sql97 <-paste('delete from   ', sourcedb,'.tbl_SRVariableDoseCDMedication where Digest is null ;', sep = "")
tb97 <- bq_project_query(project_id, sql97)



sql98 <-paste('insert into  ',targetdb,'.tbl_SRVariableDoseCDMedication
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
 Cast(left(DateEvent,19) as datetime) as DateEvent,
IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
IDMultiLexProduct,
IDMultiLexPack,
IDMultiLexDMD,
NameOfMedication,
DateMedicationStart,
DateMedicationEnd,
MedicationCourseLength,
SupervisedDose,
MedicationEndReason,
IDProfileMedicationEndedBy,
IDReferralIn,
IDEvent,
IDPatient,
sv.Digest,
IDOrganisation,
IDOrganisationRegisteredAt,
RemovedData,
cb_loadnum,
cb_exported_to_sde ,
cast(cb_rowid as int64) ,
cb_duplicate_data,
cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SRVariableDoseCDMedication sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;', sep = "")
tb98 <- bq_project_query(project_id, sql98)

eric <- paste("tbl_SRVariableDoseCDMedication done "  ,Sys.time() + hours(1) ,sep = "")
print(eric) 

#tbl_SRVisit
sql99 <-paste('delete From ', sourcedb,'.tbl_SRVisit 
where Digest is null ;', sep = "")
tb99 <- bq_project_query(project_id, sql99)




sql100 <-paste('insert into  ',targetdb,'.tbl_SRVisit
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier 
, demo.person_id 
, IDOrganisationVisibleTo
, Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded
, IDProfileEnteredBy
, cast(left(DateBooked,19) as datetime) as DateBooked
, cast(left(DateRequested,19) as datetime) as DateRequested 
, CurrentStatus
, IDProfileRequested
, IDProfileAssigned
, FollowUpDetails
, IDReferralIN
, IDPatient
, sv.Digest
, IDOrganisation
, Duration
, IDOrganisationRegisteredAt
, RemovedData
, cb_loadnum
, cb_exported_to_sde 
, cast(cb_rowid as int64)
, cb_duplicate_data
, cast(left(cb_record_added,19) as Datetime) as cb_record_added
From ', sourcedb,'.tbl_SRVisit sv
  left join ', demo, '  demo
on sv.Digest = demo.digest', sep = "")
tb100 <- bq_project_query(project_id, sql100)

eric <- paste("tbl_SRVisit done "  ,Sys.time() + hours(1) ,sep = "")
print(eric)

[1] "tbl_SRGPPracticeHistory done 2024-02-08 13:58:55.849395"
[1] "tbl_SRGoal done 2024-02-08 13:59:02.053823"
[1] "tbl_SRImmunisation done 2024-02-08 13:59:09.914019"
[1] "tbl_SRImmunisationConsent done 2024-02-08 13:59:17.129544"
[1] "tbl_SRImmunisationContent done 2024-02-08 14:59:20.862447"
[1] "tbl_SRMappingdone 2024-02-08 14:59:24.708912"
[1] "tbl_SRMappingGroup  done 2024-02-08 13:59:28.417181"
[1] "tbl_SRMedicationReadCodeDetails done 2024-02-08 13:59:33.883008"
[1] "tbl_SROnlineServices done 2024-02-08 13:59:38.727267"
[1] "tbl_SROrganisation done 2024-02-08 13:59:47.387172"
[1] "tbl_SROrganisationBranch done 2024-02-08 14:59:51.098084"
[1] "tbl_SROverseasVisitorChargingCategory done 2024-02-08 13:59:55.40949"
[1] "tbl_SRPatient done 2024-02-08 15:00:01.382296"
[1] "tbl_SRPatientAddressHistory done 2024-02-08 15:00:08.900274"
[1] "tbl_SRPatientGroups done 2024-02-08 14:00:13.354728"
[1] "tbl_SRPatientRegistration done 2024-02-08 14:00:33.385423"
[1] "tbl_SRPatientRelationship 